In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os.path import abspath, curdir, join, exists
import gffutils

In [3]:
data_dir = abspath(join(curdir, "..", "data"))
public_dir = abspath(join(curdir, "..", "frontend", "public"))

GCF_000002765.3.gff from the assembly of Plasmodium falciparum 3D7: https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_000002765.3/

In [ ]:
recreate = True
if not exists(join(public_dir, "GCF_000002765.3.db")) or recreate:
    db = gffutils.create_db(
        join(data_dir, "GCF_000002765.3.gff"),
        join(public_dir, "GCF_000002765.3.db"),
        merge_strategy="merge",
        force=True,
    )